In [23]:
import numpy as np
import numba as nb
from numba import jit
from datetime import datetime
from typing import Tuple, Mapping

Записываем функцию Матьяса и ее частные производные

In [24]:
#Функция Матьяса
def Matias_E(x,y):
  return 0.26*(x*x + y*y) - 0.48*x*y
#Производная по x
def Matias_dEdx(x,y):
  return 0.52 * x - 0.48 * y
#Производная по y
def Matias_dEdy(x,y):
  return 0.52 * y - 0.48 * x

Записываем функцию Химмельблау и ее частные производные

In [25]:
#Функция Химмельблау
def H_E(x,y):
  return (x**2+y-11)**2+(x+y**2-7)**2
#Производная по x
def H_dEdx(x,y):
  return 2*(2*x*(x**2+y-11)+x+y**2-7)
#Производная по y
def H_dEdy(x,y):
  return 2*(x**2+2*y*(x+y**2-7)+y-11)

Метод градиентного спуска из ЛР №1

In [26]:
def GradientDescent(E, dEdx, dEdy, x0, y0, LearningRate, Iterations):
  '''
  E - функция
  dEdx, dEdy - частные производные функции
  x0, y0 - начальные приближения точки
  LearningRate - скорость обучения
  Iterations - количество итераций
  '''
  #Алгоритм градиентного спуска
  x1=x0-LearningRate * dEdx(x0,y0)
  y1=y0-LearningRate * dEdy(x0,y0)
  for _ in range(Iterations):
    x0=x1
    y0=y1
    x1=x0-LearningRate * dEdx(x0,y0)
    y1=y0-LearningRate * dEdy(x0,y0)
  return x1,y1, E(x1,y1)

Тест для функции Матьяса

In [27]:
start_time_without_NUMBA_GP = datetime.now()
x, y, GPStopPoint = GradientDescent(Matias_E, Matias_dEdx, Matias_dEdy, -10, 10, 0.0001, 500000)
time_without_NUMBA_GP = datetime.now() - start_time_without_NUMBA_GP
print("Время работы без NUMBA\n", time_without_NUMBA_GP)
print('x={0}, y={1}, E={2}'.format(x,y,GPStopPoint))

Время работы без NUMBA
 0:00:00.227617
x=-1.9237412816204235e-21, y=1.9237412816204235e-21, E=3.7007805186105895e-42


Тест для функции Химмельблау

In [28]:
start_time_without_NUMBA_H = datetime.now()
x,y,HStopPoint = GradientDescent(H_E, H_dEdx, H_dEdy, -5, 5, 0.001, 500000)
time_without_NUMBA_H = datetime.now() - start_time_without_NUMBA_H
print("Время работы без NUMBA\n", time_without_NUMBA_H)
print('x={0}, y={1}, E={2}'.format(x,y,HStopPoint))

Время работы без NUMBA
 0:00:00.562816
x=-2.805118086952748, y=3.1313125182505757, E=5.837570698635487e-28


##Часть 2
Реализация градиентного спуска через NUMBA

Записываем функцию Матьяса и ее частные производные

In [29]:
#Функция Матьяса
@nb.njit(fastmath=True)
def NB_Matias_E(x: np.ndarray) -> np.float64:
  return [0.26*(x[0]*x[0] + x[1]*x[1]) - 0.48*x[0]*x[1]]
#Производная по x
@nb.njit(fastmath=True)
def NB_Matias_dEdx(x: np.ndarray) -> np.float64:
  return 0.52 * x[0] - 0.48 * x[1]
  #Производная по y
@nb.njit(fastmath=True)
def NB_Matias_dEdy(x: np.ndarray) -> np.float64:
   return 0.52 * x[1] - 0.48 * x[0]

Записываем функцию Химмельблау и ее частные производные

In [30]:
#Функция Химмельблау
@nb.njit(fastmath=True)
def NB_H_E(x: np.ndarray):
  return (x[0]**2+x[1]-11)**2+(x[0]+x[1]**2-7)**2
#Производная по x
@nb.njit(fastmath=True)
def NB_H_dEdx(x: np.ndarray):
  return 2*(2*x[0]*(x[0]**2+x[1]-11)+x[0]+x[1]**2-7)
#Производная по y
@nb.njit(fastmath=True)
def NB_H_dEdy(x: np.ndarray):
  return 2*(x[0]**2+2*x[1]*(x[0]+x[1]**2-7)+x[1]-11)

Метод градиентного спуска c NUMBA

In [31]:
@nb.njit(fastmath=True)
def NB_GradientDescent(E: Mapping, dEdx: Mapping, dEdy: Mapping, x:np.ndarray,LearningRate, Iterations):
  '''
  E - функция
  dEdx, dEdy - частные производные функции
  x - начальные приближения точки
  LearningRate - скорость обучения
  Iterations - количество итераций
  '''
  #Алгоритм градиентного спуска
  x0 = x[0];
  y0 = x[1];
  x1=x0-LearningRate * dEdx(np.array([x0, y0]))
  y1=y0-LearningRate * dEdy(np.array([x0, y0]))
  for _ in range(Iterations):
    x0=x1
    y0=y1
    x1=x0-LearningRate * dEdx(np.array([x0, y0]))
    y1=y0-LearningRate * dEdy(np.array([x0, y0]))
  return x1,y1, E(np.array([x1,y1]))

Тест для функции Матьяса

In [35]:
start_time_with_NUMBA_GP = datetime.now()
x,y,NB_GPStopPoint = NB_GradientDescent(NB_Matias_E, NB_Matias_dEdx, NB_Matias_dEdy, np.array([-10, 10]), 0.01, 50000)
time_with_NUMBA_GP = datetime.now() - start_time_with_NUMBA_GP
print("Время работы с NUMBA:\n", time_with_NUMBA_GP)
print('x={0}, y={1}, E={2}'.format(x,y,NB_GPStopPoint))

Время работы с NUMBA:
 0:00:00.010708
x=-5.693313150947087e-218, y=5.693313150947087e-218, E=[0.0]


Тест для функции Химмельблау

In [37]:
start_time_with_NUMBA_H = datetime.now()
x,y,NB_HStopPoint = NB_GradientDescent(NB_H_E, NB_H_dEdx, NB_H_dEdy, np.array([-5,5]), 0.01, 500000)
time_with_NUMBA_H = datetime.now() - start_time_with_NUMBA_H
print("Время работы с NUMBA:\n", time_with_NUMBA_H)
print('x={0}, y={1}, E={2}'.format(x,y,NB_HStopPoint))

Время работы с NUMBA:
 0:00:00.091897
x=-2.805118086952745, y=3.131312518250573, E=1.0505702830953014e-30


Сравнение времени работы программы без NUMBA и с NUMBA

In [38]:
print("Время работы программы для функции Гольдшейна-Прайса без NUMBA больше в {0} раз, по сравнению со временем работы с NUMBA.".format(round(time_without_NUMBA_GP/time_with_NUMBA_GP)))
print("Время работы программы для функции Химмельблау без NUMBA больше в {0} раз, по сравнению со временем работы с NUMBA.".format(round(time_without_NUMBA_H/time_with_NUMBA_H)))

Время работы программы для функции Гольдшейна-Прайса без NUMBA больше в 21 раз, по сравнению со временем работы с NUMBA.
Время работы программы для функции Химмельблау без NUMBA больше в 6 раз, по сравнению со временем работы с NUMBA.
